In [ ]:
!pip install langchain-openai

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.environ.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
chat = ChatOpenAI(openai_api_key=api_key, model_name="gpt-3.5-turbo")

a = llm.predict("How many planets are there?")
b = chat.predict("ow many planets are there?")

a, b

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(openai_api_key=api_key, model_name="gpt-3.5-turbo", temperature=0.1)

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?"),
]

chat.predict_messages(messages)


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(openai_api_key=api_key, model_name="gpt-3.5-turbo", temperature=0.1)

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
])

prompt = template.format_messages(
    language="Greek",
    name="Luna",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse("Hello, how, are, you")

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma speratated list of max {max_items} in lowercase. Do NOT replay with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma speratated list of max {max_items} in lowercase. Do NOT replay with anything else."),
    ("human", "{question}")
])

# Chain = tempalte + Language Model + Output Parser
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I wnat to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{receipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

final_chain = {"receipe": chef_chain} | veg_chain
final_chain.invoke({"cuisine": "indian"})

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert"),
    example_prompt,
    ("human", "What do you know about {country}?")
])


chain = final_prompt | chat
chain.invoke({
    "country": "Korea"
})

# t = PromptTemplate.from_template("What is the capital of {country}")
# t = PromptTemplate(
#     template="What is the capital of {country}",
#     input_variables=["country"],
# )
# t.format(country="France")



There are two Koreas - North Korea and South Korea. Here is some information about each:

North Korea:
Capital: Pyongyang
Language: Korean
Currency: North Korean won
Leader: Kim Jong-un

South Korea:
Capital: Seoul
Language: Korean
Currency: South Korean won
Leader: Moon Jae-in

Both countries have a rich cultural heritage and are known for their delicious cuisine, including dishes like kimchi, bibimbap, and bulgogi.

AIMessage(content='There are two Koreas - North Korea and South Korea. Here is some information about each:\n\nNorth Korea:\nCapital: Pyongyang\nLanguage: Korean\nCurrency: North Korean won\nLeader: Kim Jong-un\n\nSouth Korea:\nCapital: Seoul\nLanguage: Korean\nCurrency: South Korean won\nLeader: Moon Jae-in\n\nBoth countries have a rich cultural heritage and are known for their delicious cuisine, including dishes like kimchi, bibimbap, and bulgogi.', response_metadata={'finish_reason': 'stop'}, id='run-8663256d-35ce-489a-9f9f-c20b3f454105-0')